## 1- DataJoint & Bdata Overview

Overview tutorial for interacting with Brody Lab DataJoint SQL-based database. This notebook assumes you have followed the installation steps [here](https://github.com/Brody-Lab/bl_pipeline_python).

Written by Jess Breda February 2023

### 1. Libraries

In [3]:
#imports

In [2]:
#check connection is working

### 2. What is on DataJoint?

`Schemas` are groups of tables that are similar and the highest structure level.

**Schemas > Tables > Columns > Entries**

To read from the tables you will create a `virtual module`. This is the *whole* basis of DataJoint- connecting you from your interpreter (python, matlab etc) to your SQL database

In [8]:
# __name__ =                    (__name__, schema name)


There are nice visual ways to demonstrate relationships of tables in a `schema`, but sadly this code doesn't seem compatible with our dbs currently. Instead, let's list all the tables in each `schema`. Many will seem familiar from the old Zut webpage. 

In [1]:
## bdata_table_names = 

# print(f"Tables under bdata schema: \n {bdata_table_names[:-6]}") # removing the __vars__

In [4]:
# ratinfo_table_names = 
# print(f"Tables under ratinfo schema: \n {ratinfo_table_names[:-6]}")

### 3. Descriptive information for a table

Let's look at the descriptions (i.e. columns) for the [sessions table](https://zut-bdata00.pni.princeton.edu/sessions.php).

Let's see what the data actually looks like in the table by making a query to a specific table.

In [5]:
# query sessions

Great, we can access all the Session information. Some tables have even more descriptive information like the [Registry](https://zut-bdata00.pni.princeton.edu/rats.php) (which is stored under `Rats`)

We can even `fetch` a single columns. For example all of the experimenters that have had a rat registered to them.

In [6]:
# experimenters = 

# print(f"All Animal Experimenters: \n {np.unique(experimenters)}")

### 4. Querying tables usings keys

#### 4.1 Exact Queries
What if you don't want the whole sessions table? There are multiple levels at which you can query a table using a `key`, which is a dictionary filter for a given column. 

Let's get data for a **single session** for one of my animals `R500`. Since we are only fetching 1 thing we will call `fetch1`. You could use just `fetch` but it will add additional dimensions that are hard to get rid of.

In [7]:
session_id = '906302'
# key = 
# r500_session = 

# r500_session.keys()

In it's current form this returns a huge array. You can print it if you want but it's ugly. Instead, we can **format** our `fetch` into a `DataFrame` so it's more accessible. This is easiest to see when we fetch **all** the sessions for an animal.

In [8]:
# key = 
# r500_sessions = 

# r500_sessions.head()

This is nice because you can easily index into the table to get session level information.

In [9]:
# session_number = 

# print(f"On {r500_sessions['sessiondate'][session_number]}, {r500_sessions['n_done_trials'][session_number]} trials were done.")

Finally, lets grab all the sessions for **multiple** animals. This requires using list comprehension to make a set of `keys`.


In [10]:
# animals = 
# keys = 

# multi_animal_sessions = 

# print(f"Animal names: {multi_animal_sessions.ratname.unique()}")

#### 4.2 Logical Queries

Sometimes you might want to query information in a certain data range rather than a specific session or animal.

In [11]:
# february_mass_table = 

# print(f"{len(february_mass_table)} weighing instances this month")

You can also do ranges (using SQL syntax) and stack multiple keys.

In [13]:
# animal_count_2022 = 

# print(f"JessB had {len(animal_count_2022)} animals delivered in 2022")

For additional advanced operations (column renaming, table merging, column creation, computations across tables) see Alvaro's [notebook](https://github.com/Brody-Lab/bl_pipeline_python/blob/main/notebooks/tutorials/1-Explore%20U19%20data%20pipeline%20with%20DataJoint.ipynb). Just note the schema names have changed since he wrote it.

### 5 Exercise- Getting Water & Mass Data

Something I do nearly every day is check how much water my animals get relative to their restriction volume and where that water came from (rig or pub). 

I've written some started code to calculate the restriction volume. To complete this, we need to determine how much water was drunk in the rig versus in the pub (and where that information is stored).

#### 5.1 Restriction Target

To calculate the restriction target, you need to locate the % of body weight restriction (in `ratinfo.Registry[comments]` or `ratinfo.Water[percent_target]`) and the mass of the animal (`ratinfo.Mass[mass]`) to calculate the volume. 

For more details look in `fetch_water.py`

In [15]:
ANIMAL_ID = "R501"
DATE = "2023-02-15"

volume_target = fetch_daily_water_target(animal_id=ANIMAL_ID, date=DATE, verbose=True)

#### 5.2 Rig & Water Volumes

The goal of this exercise will be to locate where the **pub** and **rig** volumes are stored in the database.

In [18]:
# which tables?

In [16]:
# pub_keys = ### FILL IN ###
# pub_volume = ### FILL IN ###

In [17]:
# rig_keys = ### FILL IN ###
# rig_volume = ### FILL IN ###

#### 5.3 Visualize

enter the `rig_volume` and `pub_volume` amounts into the function below to run it

In [19]:
### UNCOMMENT & RUN ###
# plot_daily_water(volume_target, rig_volume, pub_volume, ANIMAL_ID, DATE)